In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
marketing_data=pd.read_csv(r'D:\analdata\marketing_data.csv')

In [ ]:
marketing_data.isna().sum()
#в колонке 'Income' 24 n/a

In [ ]:
#заметно что в тайтлах есть лишние пробелы, используем strip
marketing_data=marketing_data.rename(columns=lambda strcols: strcols.strip())
marketing_data.info() 

In [ ]:
#Income-object, нужно конвертировать в float, перед этим надо будет убрать запятую и знак $
marketing_data['Income'] = marketing_data['Income'].str.replace('$', '')
marketing_data['Income'] = marketing_data['Income'].str.replace(',', '').astype('float')

In [ ]:
marketing_data.Income=marketing_data.Income.fillna(marketing_data.Income.median()) 

In [ ]:
plt.figure(figsize=(12,6))
sns.displot(x=marketing_data.Income)

# доход +- однороден, есть немного выбросов около 150к и после 650, нагляднее будет видно на боксплоте

In [ ]:
#marketing_data['Income'].plot(kind='box', figsize=(8,6))
ax = sns.boxplot(x='Income', data=marketing_data)

In [ ]:
cols=marketing_data[['Year_Birth','Income','NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']] #

In [ ]:
#построим боксплоты для количества покупок/годов рождений,содержание покупок нас пока не интересует
fig, axes = plt.subplots(2, 3,sharey=True, figsize=(12,6))
fig.tight_layout(pad=3.0) #больше расстояния между графиками
axes=axes.flatten()
fig.suptitle('Boxplots for some numerical data')
sns.boxplot(ax=axes[0],x='Year_Birth', data=marketing_data)
sns.boxplot(ax=axes[1],x='Income', data=marketing_data)
sns.boxplot(ax=axes[2],x='NumWebPurchases', data=marketing_data)
sns.boxplot(ax=axes[3],x='NumCatalogPurchases', data=marketing_data)
sns.boxplot(ax=axes[4],x='NumStorePurchases', data=marketing_data)
sns.boxplot(ax=axes[5],x='NumWebVisitsMonth', data=marketing_data) #пока так, потом подумаю как сделать изящнее

In [ ]:
#На графиках заметны выбросы в годах рождения (скорее всего ошибка ввода), доходах(лишний ноль?), в остальных колонках их можно объяснить постоянностью некоторых покупателей

In [ ]:
#По заданию нужно определить ("What factors are significantly related to the number of store purchases"), логически наибольший вклад должен внести доход, больше денег-больше покупок, для подтверждения этой гипотезы воспользуемся простой линейной регрессией

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x=np.array(marketing_data['Income']).reshape((-1, 1))
y=np.array(marketing_data['NumStorePurchases'])
model = LinearRegression()

In [ ]:
model.fit(x, y)
r_squared = model.score(x, y)
print('coefficient of determination:', r_squared)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

In [ ]:
#действительно, почти треть

In [ ]:
#"Does US fare significantly better than the Rest of the World in terms of total purchases?" - для ответа на этот вопрос нужно будет создать новую колонку с суммой всех покупок

In [ ]:
marketing_data['TotalPurchases']=(marketing_data[['NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth']].sum(axis=1))

In [ ]:
median=marketing_data['TotalPurchases'].median()
mean=marketing_data['TotalPurchases'].mean() #среднее арифметическое и медиана +- равны, значит сильных выбросов быть не должно, удостоверимся с помощью дистплота

sns.displot(x=marketing_data['TotalPurchases'],color='grey')
plt.axvline(median, color='g', linestyle='--')
plt.axvline(mean, color='r', linestyle='-')

In [ ]:
marketing_data.groupby('Country',as_index=False).agg({'TotalPurchases':'sum'}).sort_values(by='TotalPurchases',ascending=False).reset_index(drop=True) 
#По количеству покупок US не в топе 

In [ ]:
#аналогично проверим по количеству купленных товаров
amount_columns=list(marketing_data.columns[marketing_data.columns.str.startswith('Mnt')])

In [ ]:
marketing_data['TotalAmount']=marketing_data[amount_columns].sum(axis=1)

In [ ]:
marketing_data['TotalAmount']

In [ ]:
marketing_data.groupby('Country',as_index=False).agg({'TotalAmount':'sum'}).sort_values(by='TotalAmount',ascending=False).reset_index(drop=True) 
#тут тоже не в топе, основная масса покупателей из Испании

In [ ]:
#построим графики для наглядности
df_tp=marketing_data.groupby('Country',as_index=False).agg({'TotalPurchases':'sum'}).sort_values(by='TotalPurchases', ascending=False).reset_index(drop=True)
df_ta=marketing_data.groupby('Country',as_index=False).agg({'TotalAmount':'sum'}).sort_values(by='TotalAmount', ascending=False).reset_index(drop=True)

In [ ]:
#df_ta_tp_merged=df_tp.merge(df_ta,on='Country')

In [ ]:
df_tp.plot(kind='bar',x='Country')
df_ta.plot(kind='bar',x='Country')

In [ ]:
#"Your supervisor insists that people who buy gold are more conservative."
#"Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test"
#Посмотрим на выбросы
sns.boxplot(x='MntGoldProds',data=marketing_data)

In [ ]:
import scipy.stats
scipy.stats.spearmanr(marketing_data['MntGoldProds'], marketing_data['TotalPurchases'])


In [ ]:
#Есть статистически значимая положительная корреляция
sns.lmplot(x='MntGoldProds', y='TotalPurchases', data = marketing_data);

In [ ]:
#Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish?

marketing_data[['Marital_Status','Education']]
marketing_data.Education.unique()

#Отфильтруем подходящих под "Married PhD candidates" покупателей
married_and_phd_fish=marketing_data.query('Marital_Status=="Married" and Education=="PhD"')['MntFishProducts']
married_and_not_phd=marketing_data.query('Marital_Status!="Married" and Education!="PhD"')['MntFishProducts']

#в среднем женатые и Phd покупают меньше рыбы, стат значимость проверим далее

from scipy import stats
stats.shapiro(married_and_phd_fish)

married_and_phd_fish.plot()
#H0 - в генеральной совокупности женатые и phd покупают столько же рыбопродуктов сколько и не таковые

In [ ]:
u, p_value = stats.mannwhitneyu(married_and_phd_fish, married_and_not_phd)
print ("two-sample wilcoxon-test", p_value)

In [ ]:
#Можно отклонить нулевую гипотезу, статистически значимое отличие действительно присутствует


In [ ]:
#to be continued..